In [15]:
import pandas as pd
import os
import statsmodels.api as sm
import numpy as np
data_path = "/Users/yu.zadorozhnaya/Machine-Learning-and-Predictive-Analytics-MSU-/machine_learning_01_data/"
data = pd.read_csv(os.path.join(data_path,"test_sample.csv"))

In [16]:
data.head()

,Y,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X482,X483,X484,X485,X486,X487,X488,X489,X490,X491
0,-48.041251,-0.997638,2.311532,-0.971427,-2.067563,3.446982,-1.177820,1.319834,2.135195,-2.290462,...,0.729917,-2.131048,-0.607538,0.111370,0.121842,2.887486,1.452830,-1.405036,0.352011,-1.569559
1,115.901191,-1.662864,0.753623,0.271852,0.891757,1.002363,2.472262,3.236337,1.053979,2.646357,...,-0.378999,4.141444,2.861678,-1.127431,1.426872,0.665371,4.305924,2.239984,-2.259712,0.530322
2,-112.428888,2.142538,-3.053778,-0.028428,-2.172193,4.774578,1.683675,0.058400,-0.226909,1.392892,...,-0.432519,0.312681,0.611656,-2.940966,0.451330,2.586884,0.784037,0.743123,0.269382,-1.979108
3,52.045061,-3.046474,-0.445021,-0.005266,-3.060073,-1.061151,-3.061358,1.581467,0.940564,-0.017129,...,0.597821,-1.441099,-0.662193,2.029279,-3.645747,-0.677887,-0.854942,3.873866,1.261199,1.617525
4,4.992934,0.957421,-2.825062,2.739559,1.211854,-0.334171,1.043838,-0.402130,-0.952886,0.493743,...,-2.994588,1.845936,0.842490,-0.933315,-1.381257,-3.572467,2.066330,2.935391,-3.048825,1.791418


In [17]:
Y = data.Y
X = data.drop("Y", axis = 1)
X.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X482,X483,X484,X485,X486,X487,X488,X489,X490,X491
0,-0.997638,2.311532,-0.971427,-2.067563,3.446982,-1.177820,1.319834,2.135195,-2.290462,-1.260833,...,0.729917,-2.131048,-0.607538,0.111370,0.121842,2.887486,1.452830,-1.405036,0.352011,-1.569559
1,-1.662864,0.753623,0.271852,0.891757,1.002363,2.472262,3.236337,1.053979,2.646357,-1.251855,...,-0.378999,4.141444,2.861678,-1.127431,1.426872,0.665371,4.305924,2.239984,-2.259712,0.530322
2,2.142538,-3.053778,-0.028428,-2.172193,4.774578,1.683675,0.058400,-0.226909,1.392892,-3.702617,...,-0.432519,0.312681,0.611656,-2.940966,0.451330,2.586884,0.784037,0.743123,0.269382,-1.979108
3,-3.046474,-0.445021,-0.005266,-3.060073,-1.061151,-3.061358,1.581467,0.940564,-0.017129,2.086483,...,0.597821,-1.441099,-0.662193,2.029279,-3.645747,-0.677887,-0.854942,3.873866,1.261199,1.617525
4,0.957421,-2.825062,2.739559,1.211854,-0.334171,1.043838,-0.402130,-0.952886,0.493743,0.205187,...,-2.994588,1.845936,0.842490,-0.933315,-1.381257,-3.572467,2.066330,2.935391,-3.048825,1.791418


In [18]:
def rel_imp_me(X, y): #штука, считающая вклады в R^2 разными способами
    names = X.columns
    ser = pd.Series(index = names)
    lm0 = sm.OLS(y, sm.add_constant(X)).fit()
    for c in names:
        lm = sm.OLS(y, sm.add_constant(X[names.drop(c)])).fit()
        ser[c] = lm0.rsquared - lm.rsquared 
    res = pd.DataFrame(columns =['last', 'first', 'betasq', 'pratt'], index=names)
    res['last'] = ser
    corr = X.apply(lambda x: np.corrcoef(y,x)[0,1], axis=0)
    res['first'] = corr**2
    sx = X.std()
    res['betasq'] = (lm0.params[names] * sx / np.std(y))**2
    res['pratt'] = (lm0.params[names] * sx / np.std(y)) * corr
    return res

In [19]:
import statsmodels.api as sm
fits = [sm.OLS(Y, X.iloc[:,:j]).fit() for j in range(2, 492)]
rSquared = [fit.rsquared for fit in fits] 

In [20]:
n_orig = 0
for i in range(len(rSquared)):
    if (rSquared[i] > 0.9):
        n_orig = i + 2
        break
print(n_orig)

351


In [21]:
from sklearn.decomposition import PCA
nFactors = 491
xPCA = PCA(n_components=nFactors)
xPCA.fit(X)

PCA(n_components=491)

In [22]:
factorScores = pd.DataFrame(np.dot(X, xPCA.components_.T), columns =["PC%i"%(j+1) for j in range(nFactors)])
factorScores.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC482,PC483,PC484,PC485,PC486,PC487,PC488,PC489,PC490,PC491
0,-1.976000,-3.367460,-3.151511,-1.289989,-1.707386,-2.063223,-4.622250,1.750814,4.991341,1.131704,...,0.337596,0.084279,0.166057,0.035211,-0.137388,0.061971,0.045462,-0.005665,-0.073852,0.060316
1,11.781326,-0.594073,2.542173,2.406527,0.952098,3.613517,4.230625,2.350085,-1.542905,2.426042,...,0.161113,0.085122,0.192780,-0.026779,-0.048137,0.052619,-0.063909,0.011989,-0.110128,0.052706
2,0.700326,1.204862,1.949318,5.381022,3.927573,12.265443,-5.011118,-4.388442,0.097268,-3.385767,...,0.340881,-0.017065,0.169352,-0.037612,-0.156209,0.001955,0.070182,-0.018375,-0.050273,0.065287
3,1.093012,-5.059962,5.059681,-3.056155,0.528150,-0.132542,1.633743,3.827520,6.185691,-3.243889,...,0.363868,0.116690,0.093426,-0.100426,-0.103338,0.085085,-0.026972,-0.014666,-0.057432,0.091287
4,1.214941,1.967761,3.031793,7.348924,3.817007,-0.073585,0.611276,-4.505510,-0.080461,0.346282,...,0.119340,0.088744,0.083505,-0.080438,-0.206105,0.096113,0.040528,-0.023231,-0.012741,0.051036


In [23]:
import warnings
warnings.filterwarnings('ignore')
metricsPCA = rel_imp_me(factorScores, Y)
metrics_first_rankPCA = metricsPCA["first"].rank(ascending=False, method='first')
metrics_first_rankPCA.head()

PC1     27.0
PC2     14.0
PC3    151.0
PC4    160.0
PC5     11.0
Name: first, dtype: float64

In [24]:
PC_sorted = pd.DataFrame({"Predictors" : metrics_first_rankPCA.index,
  "Rank" : metrics_first_rankPCA.values}).sort_values(by="Rank")
PC_sorted.head()

,Predictors,Rank
10,PC11,1.0
77,PC78,2.0
38,PC39,3.0
73,PC74,4.0
13,PC14,5.0


In [25]:
orderedPC = pd.DataFrame(factorScores, columns=PC_sorted["Predictors"])
orderedPC.shape

(500, 491)

In [26]:
fitsPCA = [sm.OLS(Y, orderedPC.iloc[:,:j]).fit() for j in range(2, 492)]
rSquaredPCA = [fitPCA.rsquared for fitPCA in fitsPCA] 

In [27]:
n_PCA = 0
rs = 0
for j in range(len(rSquaredPCA)):
    if (rSquaredPCA[j] > 0.9):
        n_PCA = j + 2
        rs = rSquaredPCA[j]
        break
n_PCA

137

In [28]:
print("Model dimensionality reduction:", n_orig - n_PCA, "\nDetermination coefficient:", rs)

Model dimensionality reduction: 214 
Determination coefficient: 0.9002649309423294
